# Socker Server - Framework for network servers

- sockerserver 모듈은 네트워크 서버 작성 작업을 단순화
- **4가지 서버 구상 클래스**
  - TCPServer
    - 클라이언트와 서버 간 지속적인 데이트 스트림을 제공하는 TCP 프로토콜 사용
  - UDPServer
    - 순서가 잘못되거나 전송 중 손실될 수 있는 이산적 정보 패킷인 데이터 그램을 사용
  - UnixStreamServer
  - UnixDatagramServer
    - TCP, UDP 클래스와 비슷하지만, 유닉스 도메인 소켓을 사용하는 자주 사용되지 않는 클래스
    - 유닉스 이외의 플랫폼에서는 사용할 수 없음

- 네가지 클래스는 동기적(Synchronously) 요청을 처리함
- 다음 요청을 시작하기 전 각 요청을 완료해야함
- 각 요청을 완료하는 데 시간이 오래 걸리면 적합하지 않음
- 별도의 프로세스, 스레드를 이용하면 비동기적(Asynchronously) 처리 지원
  - **ForkinMixIn**, **ThreadingMixIn** 과 같은 믹스인 클래스 사용

# 서버 생성 노트
상속 다이어그램에는 5개의 클래스가 있으며, 그 중 4개는 4가지 유형의 동기 서버를 나타냄

```
+------------+
| BaseServer |
+------------+
      |
      v
+-----------+        +------------------+
| TCPServer |------->| UnixStreamServer |
+-----------+        +------------------+
      |
      v
+-----------+        +--------------------+
| UDPServer |------->| UnixDatagramServer |
+-----------+        +--------------------+
```

아래의 믹스인 클래스를 사용하여 각 서버의 포킹(forking)과 스레딩(threading) 버전을 만들 수 있음.

class socketserver.**ForkingMixIn**

class socketserver.**ThreadingMixIn**

## ThreadingUDPServer 생성

In [3]:
import socketserver

# ThreadingUDPServer
class ThreadingUDPServer(socketserver.ThreadingMixIn, socketserver.UDPServer):
  pass

- 서비스를 구현하기 위해선 **BaseRequestHandler**에서 클래스를 파생시키고 **handle()** 메소드를 오버라이딩 해야함
- 서버 클래스 중 하나와 요청 처리기 클래스를 결합하여 다양한 버전의 서비스 실행 가능

# 서버 객체 & 요청 처리기 객체
https://docs.python.org/ko/3/library/socketserver.html#server-creation-notes 참고

# TCP server 구현 예시

In [7]:
import socketserver

class MyTCPHandler(socketserver.BaseRequestHandler):
  """
  The request handler class for our server.
  
  It is instantiated onece per connection to the server, and must
  override the handle() method to implement communication to the client.
  """
  
  # handle() 메소드 오버라이딩 
  def handle(self):
    # self.request는 클라이언트에 연결된 TCP 소켓
    self.data = self.request.recv(1024).strip()
    print(f"Recieved from {self.client_address[0]}")
    print(self.data)
    
    # 동일한 데이터를 대문자로 다시 보냄
    self.request.sendall(self.data.upper())
    
if __name__ == "__main__":
  HOST, PORT = "localhost", 9999
  
  # 서버를 생성하고 localhost:9999 를 바인딩
  # 서버 활성화, Ctrl-C : 서버 종료
  with socketserver.TCPServer((HOST, PORT), MyTCPHandler) as server:
    server.serve_forever()

Recieved from 127.0.0.1
b'GET / HTTP/1.1\r\nHost: localhost:9999\r\nConnection: keep-alive\r\nCache-Control: max-age=0\r\nsec-ch-ua: "Google Chrome";v="125", "Chromium";v="125", "Not.A/Brand";v="24"\r\nsec-ch-ua-mobile: ?0\r\nsec-ch-ua-platform: "Windows"\r\nUpgrade-Insecure-Requests: 1\r\nUser-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36\r\nAccept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7\r\nSec-Fetch-Site: none\r\nSec-Fetch-Mode: navigate\r\nSec-Fetch-User: ?1\r\nSec-Fetch-Dest: document\r\nAccept-Encoding: gzip, deflate, br, zstd\r\nAccept-Language: ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7\r\nCookie: username-localhost-8888=2|1:0|10:1719279671|23:username-localhost-8888|188:eyJ1c2VybmFtZSI6ICIwZDY3Nzc5NGZjYTE0ZjgzOWQwYzU1MDgxNDAxMjVmOCIsICJuYW1lIjogIkFub255bW91cyBLb3JlIiwgImRpc3BsYXlfbmFtZSI6ICJBbm9ueW1vdXMgS29yZSIsICJpbm

KeyboardInterrupt: 

In [5]:
# 스트림(표준 파일 인터페이스를 제공하여 통신을 단순화하는 파일류 객체)을 사용하는 대체 요청 처리기 클래스
class MyTCPHandler(socketserver.StreamRequestHandler):

    def handle(self):
        # self.rfile is a file-like object created by the handler;
        # we can now use e.g. readline() instead of raw recv() calls
        self.data = self.rfile.readline().strip()
        print("{} wrote:".format(self.client_address[0]))
        print(self.data)
        # Likewise, self.wfile is a file-like object used to write back
        # to the client
        self.wfile.write(self.data.upper())

In [ ]:
# 클라이언트

import socket
import sys

HOST, PORT = "localhost", 9999
data = " ".join(sys.argv[1:])

# Create a socket (SOCK_STREAM means a TCP socket)
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
    # Connect to server and send data
    sock.connect((HOST, PORT))
    sock.sendall(bytes(data + "\n", "utf-8"))

    # Receive data from the server and shut down
    received = str(sock.recv(1024), "utf-8")

print("Sent:     {}".format(data))
print("Received: {}".format(received))

ConnectionRefusedError: [WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다

# 비동기 믹스인 구현

In [9]:
import socket
import threading
import socketserver

class ThreadedTCPRequestHandler(socketserver.BaseRequestHandler):

    def handle(self):
        data = str(self.request.recv(1024), 'ascii')
        cur_thread = threading.current_thread()
        response = bytes("{}: {}".format(cur_thread.name, data), 'ascii')
        self.request.sendall(response)

class ThreadedTCPServer(socketserver.ThreadingMixIn, socketserver.TCPServer):
    pass

def client(ip, port, message):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
        sock.connect((ip, port))
        sock.sendall(bytes(message, 'ascii'))
        response = str(sock.recv(1024), 'ascii')
        print("Received: {}".format(response))

if __name__ == "__main__":
    # Port 0 means to select an arbitrary unused port
    HOST, PORT = "localhost", 0

    server = ThreadedTCPServer((HOST, PORT), ThreadedTCPRequestHandler)
    with server:
        ip, port = server.server_address

        # Start a thread with the server -- that thread will then start one
        # more thread for each request
        server_thread = threading.Thread(target=server.serve_forever)
        # Exit the server thread when the main thread terminates
        server_thread.daemon = True
        server_thread.start()
        print("Server loop running in thread:", server_thread.name)

        client(ip, port, "Hello World 1")
        client(ip, port, "Hello World 2")
        client(ip, port, "Hello World 3")

        server.shutdown()

Server loop running in thread: Thread-3 (serve_forever)
Received: Thread-4 (process_request_thread): Hello World 1
Received: Thread-5 (process_request_thread): Hello World 2
Received: Thread-6 (process_request_thread): Hello World 3
